In [7]:
import requests
from io import StringIO
import pandas as pd
import numpy as np
from tqdm import tqdm
import datetime
import time

/home/acer/.pyenv/versions/py368/lib/python3.6/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [8]:
import smtplib, ssl
def connection():
    email_smtp_port = 465  # For SSL
    email_smtp_server = "smtp.gmail.com"
    email_sender_username = "robottoolkit@gmail.com"
    email_sender_password = "!Robotify"
    # Create a secure SSL context
    context = ssl.create_default_context()
    server = smtplib.SMTP_SSL(email_smtp_server, email_smtp_port, context=context)
    server.login(email_sender_username, email_sender_password)
    return server

sender_email = email_sender_username
receiver_email = "oo24514@gmail.com"


In [9]:
import jupyternotify
ip = get_ipython()
# ip.register_magics()
jnotify = jupyternotify.JupyterNotifyMagics(ip)

<IPython.core.display.Javascript object>

In [ ]:
target = 29.50
minimum = 100.0
maximum = 0.0
start = time.time()
while True:
    r = requests.get('https://www.fubon.com/Fubon_Portal/banking/Personal/deposit/exchange_rate/exchange_rate1.jsp')
    exchange = float(pd.read_html(StringIO(r.text))[0].loc[0]['即期銀行買入銀行賣出'].split(' ')[-1])
    if exchange <= target:
        info = f"!!!!!{exchange}"
        jnotify.notify(f"-m {info}")
        
        message = f"""\
        Subject: Exchange

        target {exchange} 
        minimum {minimum} 
        maximum {maximum}.
        """
        if minimum > exchange:
            try:
                server.sendmail(sender_email, receiver_email, message)
            except:
                server = connection()
                server.sendmail(sender_email, receiver_email, message)
    if exchange < minimum:
        info = f"Minimum,{exchange}"
        jnotify.notify(f"-m {info}")
        minimum = exchange
    if exchange > maximum:
        info = f"Maximum,{exchange}"
        jnotify.notify(f"-m {info}")
        maximum = exchange
    time.sleep(10)
    if time.time() - start > 60:
        jnotify.notify(f"-m {exchange}_{minimum-exchange:.4f}_{maximum-exchange:.4f}")
        start = time.time()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
29.5315*200

5906.3

In [ ]:
import finlab

In [ ]:
df = finlab.data.get("每股盈餘")

In [ ]:
df.loc['2019-08':]['3046'].sum()

In [ ]:
for col in df.columns:
    if len(col) > 4:
        print(col)

In [ ]:
finlab.backtest(5)

In [ ]:
dir(finlab)

In [ ]:
https://www.fubon.com/Fubon_Portal/banking/Personal/deposit/exchange_rate/exchange_rate1.jsp

In [ ]:
 
import notify2

def popUpMessage(title, message):
    print("Initializing pynotify")
    pynotify.init("Scorer")
    pynotify.Notification(title, message, "dialog-information").show()

In [ ]:
popUpMessage("13", "buybuybuy")

In [ ]:
r.text.replace("\t", "").split("\n")

In [ ]:
def Day_Stock(date_str):
    r = requests.post('https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + date_str + '&type=ALL')
    df = pd.read_csv(StringIO(r.text.replace("=", "")), header=["證券代號" in l for l in r.text.split("\n")].index(True)-1)
    df = df.apply(lambda s: s.astype(str).str.replace(",", "").replace("+", "1").replace("-", "-1"))
    df = df.set_index("證券代號")
    df.drop("Unnamed: 16", axis=1, inplace=True)
    df.drop(index=[col for col in df.index if len(col) > 4], errors='ignore', inplace=True)
    return df

def to_timeseries(df_dict, colname):
    df_timeseries = []
    for target_date_str in df_dict:
        target_df = df_dict[target_date_str][[colname]].transpose()
        target_df = target_df.rename(index={colname: target_date_str})
        target_df.set_index(pd.to_datetime(target_df.index))
        df_timeseries.append(target_df)
    df_timeseries = pd.concat(df_timeseries)
    df_timeseries = df_timeseries.set_index(pd.to_datetime(df_timeseries.index))
    return df_timeseries

def save_daily_data(start, num_day=None, pack=0, merge=True):
    now = datetime.datetime.now()
    y, m, d = [i for i in map(int, start.split("-"))]
    start = datetime.datetime(y, m, d)
    if num_day is None:
        num_day = max(now.day - start.day, 1)
    DAY = datetime.timedelta(1)
    dfs = {}
    for i in range(num_day):
        progress = '.' * (i + 1)
        print(f"\rCrawling {progress}", end="")
        zzz = np.random.randint(5,10)
        time.sleep(zzz)
        target_date = start + i * DAY
        if target_date > now:
            break
        weekday = target_date.strftime('%w')
        target_date_str = target_date.strftime('%Y%m%d')
        if weekday not in ['0', '6']:
            if target_date == now and now.hour <= 14:
                continue
            try:
                _df = Day_Stock(target_date_str)
            except ValueError:
                print(target_date_str)
                continue
            dfs[target_date_str] = _df
    total_df = []
    for col in ['最高價', '最低價', '收盤價', '本益比', '成交股數']:
        print(f"\rSaving {col}                      ", end="")
        if pack:
            fname = os.path.join(os.getcwd(), "..", "data", f"{col}{pack:03}.pkl")
        else:
            fname = os.path.join(os.getcwd(), "..", "data", f"{col}.pkl")
        df = to_timeseries(dfs, col)
        if os.path.exists(fname) and merge:
            _df = pd.read_pickle(fname)
            df = pd.concat([df, _df])
        df.drop_duplicates().to_pickle(fname)

def get_data(name, pack=0):
    if pack:
        fname = os.path.join(os.getcwd(), "..", "data", f"{name}{pack:03}.pkl")
    else:
        fname = os.path.join(os.getcwd(), "..", "data", f"{name}.pkl")
    assert os.path.exists(fname), f"{fname} does not exist."
    df = pd.read_pickle(fname)
    return df


In [ ]:
temp = get_data('本益比')
temp

In [ ]:
save_daily_data('2020-06-10', 1)

In [ ]:
pd.read_pickle(fname)

In [ ]:
start = datetime.datetime(2020,6,1) + 5 * DAY
start

In [ ]:
start = datetime.datetime(2015,1,1)

In [ ]:
start - datetime.datetime.now()

In [ ]:
start = datetime.datetime(2015,1,1)
DAY = datetime.timedelta(1) 
num_day = 1
pack = 1
while start <= datetime.datetime.now():
    # print(start)
    if (num_day % 39 == 0 and num_day != 0):
        # print(start - (num_day-1) * DAY)
        save_daily_data((start - (num_day+1) * DAY).strftime('%Y-%m-%d'), num_day=num_day, pack=pack, merge=False)
        num_day = 0
        pack += 1
    start += DAY
    
    num_day += 1


In [ ]:
start.strftime('%Y%m%d')

In [ ]:
start + datetime.timedelta(1) * (end - start).days

In [ ]:
start = '2020-06-15'
start = start.split("-")

In [ ]:
[i for i in map(int, start)]

In [ ]:
import os

In [ ]:
root = os.getcwd()
root

In [ ]:
for df, col in zip(total_df, ['最高價', '最低價', '收盤價', '本益比', '成交股數']):
    df.to_pickle(os.path.join(os.getcwd(), "..", "data", f"{col}.pkl"))

In [ ]:
df = pd.read_pickle(os.path.join(os.getcwd(), "..", "data", f"{col}.pkl"))

In [ ]:
df

In [ ]:
df.set_index(pd.to_datetime(df.index))

In [ ]:
peratio

In [ ]:
pd.to_datetime(peratio.index)

In [ ]:
peratio = dfs[target_date_str][['本益比']].transpose()
peratio = peratio.rename(index={'本益比': target_date_str})
peratio.set_index(pd.to_datetime(peratio.index))

In [ ]:
peratio = dfs[target_date_str][['本益比']].transpose()

In [ ]:
target = {}
for date in dfs:
    target[date] = dfs[date].loc['0050']

In [ ]:
target_df = pd.DataFrame(target).transpose()

In [ ]:
target_df.set_index(pd.to_datetime(target_df.index))

In [ ]:


datestr = datetime.datetime.now().strftime('%Y%m%d')

# 下載股價
r = requests.post('https://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + datestr + '&type=ALL')



In [ ]:
# 整理資料，變成表格
df = pd.read_csv(StringIO(r.text.replace("=", "")), 
            header=["證券代號" in l for l in r.text.split("\n")].index(True)-1)

# 整理一些字串：
df = df.apply(lambda s: s.astype(str).str.replace(",", "").replace("+", "1").replace("-", "-1"))

# 顯示出來


In [ ]:
df = df.set_index("證券代號")
df.drop("Unnamed: 16", axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df = df.astype(
    {
        '成交股數': np.float,
        '成交筆數': np.float,
        '成交金額': np.float,
        '開盤價': np.float,
        '最高價': np.float,
        '最低價': np.float,
        '收盤價': np.float,
        '漲跌(+/-)': np.float,
        '漲跌價差': np.float,
        '最後揭示買價': np.float,
        '最後揭示買量': np.float,
        '最後揭示賣價': np.float,
        '最後揭示賣量': np.float,
        '本益比': np.float
    },
    errors='ignore')

In [ ]:
df[pd.to_numeric(df['本益比'], errors='coerce')<15]